In [58]:
import os

all_paths = []
for root, dirs, files in os.walk("./processed_data"):
    for file in files:
        if file.endswith(".txt"):
             all_paths.append(os.path.join(root, file))

In [78]:
import sentencepiece as spm
import os

data = ",".join(all_paths)

options = dict(
    input=data,
    model_prefix='tokenicer',
    model_type='bpe',
    vocab_size=3000,
    normalization_rule_name='identity',
    remove_extra_whitespaces=False,
    input_sentence_size=2*1e6,
    max_sentence_length=4192,
    seed_sentencepiece_size=1e5,
    shuffle_input_sentence=True,
    #rare word treatment
    character_coverage=0.9995,
    byte_fallback=True,
    #merge rules
    split_digits=True,
    split_by_unicode_script=True,
    split_by_whitespace=True,
    split_by_number=True,
    max_sentencepiece_length=16,
    add_dummy_prefix=True,
    allow_whitespace_only_pieces=True,
    #special tokens
    unk_id=0, # the unk token MUST exist
    bos_id=1,
    eos_id=2,
    pad_id=-1,
    # system
    num_threads=os.cpu_count(), # use all cores
)

spm.SentencePieceTrainer.train(**options)

In [79]:
# Generates a dictionary mapping unique texts/tokens with their corresponding color
def get_token_colors(text, tokenizer):
    tokens = tokenizer.tokenize(text)
    unique_tokens = list(set(tokens))
    token_colors = {token: number_to_color(index) for index, token in enumerate(unique_tokens)}
    return token_colors

def get_token_colors_sentencepiece(text, tokenizer):
    tokens = tokenizer.encode_as_pieces(text)
    unique_tokens = list(set(tokens))
    token_colors = {token: number_to_color(index) for index, token in enumerate(unique_tokens)}
    return token_colors


# Displays the tokens with the unique color
def display_colored_tokens(tokens, token_colors):
    colored_text = ""
    for token in tokens:
        color = token_colors[token]
        colored_text += f'<span style="background-color: {color}; padding: 2px;">{token}</span> '

    display(HTML(colored_text))

# Generates a unique HSL color based on the index of the token
def number_to_color(number):
    golden_ratio_conjugate = 0.618033988749895
    a = 1664525
    c = 1013904223
    m = 2**32

    pseudorandom = (a * number + c) % m
    hue = ((pseudorandom * golden_ratio_conjugate) % 1) * 360
    s = 60 + (pseudorandom % 21)
    l = 70 + (pseudorandom % 21)

    return f"hsl({hue}, {s}%, {l}%)"

In [83]:
text = "Nói albinói er fyrsta kvikmynd Dags Kára Péturssonar í fullri lengd. Hún fjallar um ungan dreng að nafni Nói, sem býr í afskekktum bæ á Íslandi. Nói á í erfiðleikum í skóla og fær litla virðingu heima hjá sér. Hann kynnist ungri stelpu frá Reykjavík og ákveður að strjúka í burtu með henni. En hún er ekki á sama máli. Kvikmyndin hlaut sex Edduverðlaun 2003 og var send í forval til Óskarsins 2004. "

In [84]:
from IPython.display import HTML, display
import json

sp = spm.SentencePieceProcessor()
sp.load('tokenicer.model')

tokens = sp.encode_as_pieces(text)

token_colors = get_token_colors_sentencepiece(text, sp)

display(HTML(f"<p><b>Model:</b> tokenICEr</p><p><b>Tokens:</b>"))
display_colored_tokens(tokens, token_colors)

# Calculate and display the number of tokens and characters
num_tokens = len(tokens)
num_chars = len(text)
print(f"Number of Tokens: {num_tokens}")
print(f"Number of Characters: {num_chars}")
print("="*50)

Number of Tokens: 162
Number of Characters: 399


In [85]:

# importing from huggingface transformers
from transformers import AutoTokenizer
from IPython.display import HTML, display
import json


# Display tokens with colors for each model
model = 'openai-community/gpt2'

tokz = AutoTokenizer.from_pretrained(model)
token_colors = get_token_colors(text, tokz)
tokens = tokz.tokenize(text)

display(HTML(f"<p><b>Model:</b> {model}</p><p><b>Tokens:</b>"))
display_colored_tokens(tokens, token_colors)

# Calculate and display the number of tokens and characters
num_tokens = len(tokens)
num_chars = len(text)
print(f"Number of Tokens: {num_tokens}")
print(f"Number of Characters: {num_chars}")
print("="*50)

# Additional code for raw tokens
#raw_tokens = [tokz.tokenize(text) for tokz in [AutoTokenizer.from_pretrained(model) for model in model_nm]]
#raw_tokens_json = json.dumps(raw_tokens, indent=2)
#print(f"Raw Tokens: {raw_tokens_json}")

Number of Tokens: 203
Number of Characters: 399
